In [74]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as dset

import matplotlib.pyplot as plt
import pickle
import os

## Dataloader

In [70]:
class MultipleSourceDataSet(Dataset):
    
    def __init__(self, json_file, root_dir, transform = None):
        
        with open(root_dir + 'block0.json') as f:
            self.result = torch.Tensor(json.load(f))
            
        self.root_dir = root_dir
        self.transform = transform
        
    def __len__(self):
        return len(self.result[0])
    
    def __getitem__(self):
        None

In [ ]:
trainloader = 

In [71]:
import json
root_dir = '/home/tk/Documents/blocks/'
with open (root_dir + 'block0.json') as f:
    result = json.load(f)
    
with open (root_dir + 'block1.json') as f:
    result1 = json.load(f)

In [85]:
len(x[0][0])

1025

In [17]:
trans = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5,), (1.0,))
    ])

trainset = dset.MNIST(root = '../data', train = True, transform = trans, download = True)
testset = dset.MNIST(root = '../data', train = False, transform = trans)

In [18]:
trainloader = torch.utils.data.DataLoader(dataset = trainset,
                                                batch_size = 8,
                                                shuffle = True)

testloader = torch.utils.data.DataLoader(dataset = testset,
                                               batch_size = 8,
                                               shuffle = True)

In [24]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28*28, 500)
        self.fc2 = nn.Linear(500, 256)
        self.fc3 = nn.Linear(256, 10)

        
    def forward(self, x):
        x = x.view(-1, 28*28)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x
    
model = Net()
print (model)

Net(
  (fc1): Linear(in_features=784, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=10, bias=True)
)


In [36]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.008, momentum = 0.03)

In [ ]:
model.train()
for epoch in range(10):
    running_loss = 0
    
    for i, data in enumerate(trainloader, 0):
        
        inputs, labels = data
#         print (data[0].shape)
#         print (data[1].shape)
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
#         if i % 2000 == 0:
#             print (loss.item())
        running_loss += loss.item()
        if i % 1000 == 0:
            print ('[%d, %5d] loss: %.3f' % (epoch, i, running_loss/ 1000))
            running_loss = 0
            
torch.save(model, 'FeatureNet.pkl')